# Проект по SQL.

### Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

### Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Импорт библиотек.

In [15]:
import pandas as pd
from sqlalchemy import create_engine 

## Установка парметров.

In [16]:
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных

In [17]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

### Сохраняем коннектор.

In [18]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Чтобы выполнить SQL-запрос, используем Pandas:

pd.io.sql.read_sql(query, con = engine)

## Исследование таблицы.

In [21]:
query_1 = '''
         SELECT *
         FROM books
         LIMIT 5
        '''
books = pd.io.sql.read_sql(query_1, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Исследуем первые 5 строк таблицы "authors":

In [22]:
query_2 = '''
         SELECT *
         FROM authors
         LIMIT 5
        '''
authors = pd.io.sql.read_sql(query_2, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Исследуем первые 5 строк таблицы "publishers":

In [23]:
query_3 = '''
         SELECT *
         FROM publishers
         LIMIT 5
        '''
publishers = pd.io.sql.read_sql(query_3, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Исследуем первые 5 строк таблицы "ratings":

In [24]:
query_4 = '''
         SELECT *
         FROM ratings
         LIMIT 5
        '''
ratings = pd.io.sql.read_sql(query_4, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Исследуем первые 5 строк таблицы "reviews":

In [25]:
query_5 = '''
         SELECT *
         FROM reviews
         LIMIT 5
        '''
reviews = pd.io.sql.read_sql(query_5, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


С таблицами всё в порядке, названия и отображения столбцов в норме. Можно работать.

## Cколько книг вышло после 1 января 2000 года;

In [26]:
count_book = '''
         SELECT 
               COUNT(book_id)
         FROM  books
         WHERE 
               publication_date >= '01-01-2000'
        '''
books_after_2000 = pd.io.sql.read_sql(count_book, con = engine)
books_after_2000

,count
0,821


После 1 января 2000 года вышло 821 книга.

## Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [27]:
count_rating = '''
            SELECT
                books.title,
                (
                    SELECT
                        COUNT(review_id) AS review_count
                    FROM reviews
                    WHERE book_id = books.book_id
                ),
                (
                    SELECT
                        AVG(rating)
                    FROM ratings
                    WHERE book_id = books.book_id) AS avg_rating
            FROM books, reviews
            GROUP BY books.book_id
            ORDER BY review_count DESC
            
        '''

reviews_and_average_rating = pd.io.sql.read_sql(count_rating, con=engine)


reviews_and_average_rating

,title,review_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


Топ-5 книг с самой высоким количеством обзоров и средней оценкой.

   1.Сумерки
  
   2.Гарри Поттер и узник Азкабана
  
   3.Гарри Поттер и тайная комната
  
   4.Воровка книг
  
   5.Стеклянный замок

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц;

In [28]:
publishers_max = '''
         SELECT
               publishers.publisher_id,
               publishers.publisher AS publisher_name,
               COUNT(books.book_id) AS count_book
               
         FROM  books
               LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
         WHERE 
               books.num_pages > 50
         GROUP BY
               publisher_name,
               publishers.publisher_id
         ORDER BY 
               count_book DESC
         LIMIT 1;      
        '''
publishing_house= pd.io.sql.read_sql(publishers_max, con = engine)
publishing_house

,publisher_id,publisher_name,count_book
0,212,Penguin Books,42


Издательство Penguin Books - 42 произведения толще 50 страниц.

## Определим автора с самой высокой средней оценкой книг;

In [29]:
avg_max_rating =  '''
          SELECT
              author,
              AVG(sub.avg_rating) AS high_avg_rating
          FROM
              (SELECT
                  author,
                  books.book_id,
                  AVG(rating) AS avg_rating
              FROM 
                  authors
              LEFT JOIN books ON books.author_id = authors.author_id
              LEFT JOIN ratings ON ratings.book_id = books.book_id
              GROUP BY
                  author,
                  books.book_id
              HAVING
                  COUNT(rating) > 50) AS sub
          GROUP BY
              author
          ORDER BY
              high_avg_rating DESC
        '''

author_with_the_highest_rating = pd.io.sql.read_sql(avg_max_rating, con = engine)
author_with_the_highest_rating

,author,high_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Топ-5 авторов с самой высокой оценкой:
    
1.Джоан Роулинг - Британская писательница

2.Маркус Зузак - Австралийский писатель

3.Джон Рональд Руэл Толкин - Британский писатель

4.Рик Риордан - Американский писатель

5.Уильям Голдинг - Британский писатель

В итоге можно наблюдать что из 5 писателей с самой высокой средней оценкой 3 являются гражданами Великобритании.

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [30]:
review_user = '''

        SELECT
            AVG(sub_review_cnt.review_cnt) AS avg_review_cnt
        FROM
            (SELECT
                COUNT(text) AS review_cnt
            FROM 
                reviews
            WHERE username IN
                (SELECT username
                FROM 
                    ratings
                GROUP BY
                    username
                HAVING
                    COUNT(ratings) > 50)
            GROUP BY 
                    username
            ) AS sub_review_cnt  
            
        '''
        
avg_count_review = pd.io.sql.read_sql(review_user, con = engine)
avg_count_review

,avg_review_cnt
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24 обзора.

# Выводы:

1. После 1 января 2000 года вышло 821 книга.


2. Топ-5 книг с самой высоким количеством обзоров и средней оценкой.

   1.Сумерки
  
   2.Гарри Поттер и узник Азкабана
  
   3.Гарри Поттер и тайная комната
  
   4.Воровка книг
  
   5.Стеклянный замок
   
  
3. Издательство, которое выпустило наибольшее число книг толще 50 страниц.
   
   Британско издательство Penguin Books.
   
   
4. Авторы с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками.

   1.Джоан Роулинг - Британская писательница

   2.Маркус Зузак - Австралийский писатель

   3.Джон Рональд Руэл Толкин - Британский писатель

   4.Рик Риордан - Американский писатель

   5.Уильям Голдинг - Британский писатель
   
   
5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24 обзора.